# <img src="https://kaggle2.blob.core.windows.net/competitions/kaggle/4594/logos/front_page.png"/><span style="color:blue;text-align:center;">v3 Feature Engineering</span>

Rossmann operates over 3,000 drug stores in 7 European countries. Currently, 
Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied. Reliable sales forecasts enable store managers to create effective staff schedules that increase productivity and motivation. By helping Rossmann create a robust prediction model, you will help store managers stay focused on what’s most important to them: their customers and their teams! 
<img src="https://kaggle2.blob.core.windows.net/competitions/kaggle/4594/media/rossmann_banner2.png"/>

In [1]:
using DataFrames
using MLBase
using GLM
using Gadfly
using Iterators

  likely near /Users/diego/.julia/v0.4/MLBase/src/modeltune.jl:5
  likely near /Users/diego/.julia/v0.4/MLBase/src/modeltune.jl:5
  likely near /Users/diego/.julia/v0.4/MLBase/src/modeltune.jl:5
  likely near /Users/diego/.julia/v0.4/MLBase/src/deprecated/datapre.jl:104
  likely near /Users/diego/.julia/v0.4/MLBase/src/deprecated/datapre.jl:105
  likely near /Users/diego/.julia/v0.4/MLBase/src/deprecated/datapre.jl:163
  likely near /Users/diego/.julia/v0.4/MLBase/src/deprecated/datapre.jl:163
  likely near /Users/diego/.julia/v0.4/MLBase/src/deprecated/datapre.jl:163


## Load Data

In [2]:
train = readtable("data/train.csv")
test = readtable("data/test.csv")
store = readtable("data/store.csv")
train = join(train, store, on=:Store)
test = join(test, store, on=:Store)
all_data = vcat(train, test);

## Sample Data Visualization

In [3]:
showcols(train)
showcols(test)

1017209x18 DataFrames.DataFrame
| Col #

## Initial Preprocessing

### Select Initial Features and Label

In [4]:
initial_features = [:DayOfWeek, :Open, :Promo, :StateHolidayEnc, :SchoolHoliday, 
                    :StoreTypeEnc, :AssortmentEnc, :Promo2, :PromoIntervalEnc]
label = :Sales;

### Encode Categorical Fields

In [5]:
label_state_holiday = labelmap(vcat(train[:StateHoliday], test[:StateHoliday]))
label_store_type = labelmap(vcat(train[:StoreType], test[:StoreType]))
label_assortment = labelmap(vcat(train[:Assortment], test[:Assortment]))
label_promo_interval = labelmap(vcat(train[:PromoInterval], test[:PromoInterval]));

In [6]:
train[:StateHolidayEnc] = labelencode(label_state_holiday, train[:StateHoliday])
train[:StoreTypeEnc] = labelencode(label_store_type, train[:StoreType])
train[:AssortmentEnc] = labelencode(label_assortment, train[:Assortment])
train[:PromoIntervalEnc] = labelencode(label_promo_interval, train[:PromoInterval])

test[:StateHolidayEnc] = labelencode(label_state_holiday, test[:StateHoliday])
test[:StoreTypeEnc] = labelencode(label_store_type, test[:StoreType])
test[:AssortmentEnc] = labelencode(label_assortment, test[:Assortment])
test[:PromoIntervalEnc] = labelencode(label_promo_interval, test[:PromoInterval])

all_data[:StateHolidayEnc] = labelencode(label_state_holiday, all_data[:StateHoliday])
all_data[:StoreTypeEnc] = labelencode(label_store_type, all_data[:StoreType])
all_data[:AssortmentEnc] = labelencode(label_assortment, all_data[:Assortment])
all_data[:PromoIntervalEnc] = labelencode(label_promo_interval, all_data[:PromoInterval]);

 | Name                      | Eltype     | Missing |
|-------|---------------------------|------------|---------|
| 1     | Store                     | Int64      | 0       |
| 2     | DayOfWeek                 | Int64      | 0       |
| 3     | Date                      | UTF8String | 0       |
| 4     | Sales                     | Int64      | 0       |
| 5     | Customers                 | Int64      | 0       |
| 6     | Open                      | Int64      | 0       |
| 7     | Promo                     | Int64      | 0       |
| 8     | StateHoliday              | UTF8String | 0       |
| 9     | SchoolHoliday             | Int64      | 0       |
| 10    | StoreType                 | UTF8String | 0       |
| 11    | Assortment                | UTF8String | 0       |
| 12    | CompetitionDistance       | Int64      | 2642    |
| 13    | CompetitionOpenSinceMonth | Int64      | 323348  |
| 14    | CompetitionOpenSinceYear  | Int64      | 323348  |
| 15    | Promo2               

In [7]:
showcols(all_data)

1058297x23 DataFrames.DataFrame


### Handling Missing Data

#### Open

In [8]:
test[isna(test[:Open]), :Open] = 1
all_data[isna(all_data[:Open]), :Open] = 1;

#### Competition Distance

In [9]:
function lm_for_prediction(data, y, x)
    features = vcat(x, y)
    formula = eval(parse(string(y , " ~ ", join(x, " + "))))
    return glm(formula, data[!isna(data[y]), :], Normal(), IdentityLink());
end

function apply_lm_to_data(lm, features, new_feature)
    train[new_feature] = predict(lm, train)
    test[new_feature] = predict(lm, test)
    all_data[new_feature] = predict(lm, all_data);
end

| Col # | Name                      | Eltype     | Missing |
|-------|---------------------------|------------|---------|
| 1     | Store                     | Int64      | 0       |
| 2     | DayOfWeek                 | Int64      | 0       |
| 3     | Date                      | UTF8String | 0       |
| 4     | Sales                     | Int64      | 41088   |
| 5     | Customers                 | Int64      | 41088   |
| 6     | Open                      | Int64      | 11      |
| 7     | Promo                     | Int64      | 0       |
| 8     | StateHoliday              | UTF8String | 0       |
| 9     | SchoolHoliday             | Int64      | 0       |
| 10    | StoreType                 | UTF8String | 0       |
| 11    | Assortment                | UTF8String | 0       |
| 12    | CompetitionDistance       | Int64      | 2738    |
| 13    | CompetitionOpenSinceMonth | Int64      | 338564  |
| 14    | CompetitionOpenSinceYear  | Int64      | 338564  |
| 15    | Promo2        

apply_lm_to_data (generic function with 1 method)

In [10]:
lm = lm_for_prediction(all_data, :CompetitionDistance, initial_features);

In [11]:
apply_lm_to_data(lm, vcat(initial_features, :CompetitionDistance), :CompetitionDistanceHM);

#### Competition Open Since Month

In [12]:
lm = lm_for_prediction(all_data, :CompetitionOpenSinceMonth, initial_features);

In [13]:
apply_lm_to_data(lm, vcat(initial_features, :CompetitionOpenSinceMonth), :CompetitionOpenSinceMonthHM);

#### Competition Open Since Year

In [14]:
features = [:DayOfWeek, :Open, :Promo, :Promo2, 
            :StateHolidayEnc, :StoreTypeEnc, :AssortmentEnc, 
            :PromoIntervalEnc, :CompetitionDistanceHM]
lm = lm_for_prediction(all_data, :CompetitionOpenSinceYear, features);

In [15]:
apply_lm_to_data(lm, vcat(features, :CompetitionOpenSinceYear), :CompetitionOpenSinceYearHM);

#### Promo 2 Since Week

In [16]:
features = [:Promo, :StateHolidayEnc, :StoreTypeEnc, :AssortmentEnc, :PromoIntervalEnc]
lm = lm_for_prediction(all_data, :Promo2SinceWeek, features);

In [17]:
apply_lm_to_data(lm, vcat(features, :Promo2SinceWeek), :Promo2SinceWeekHM);

#### Promo 2 Since Year

In [18]:
features = [:Promo, :StateHolidayEnc, :StoreTypeEnc, :AssortmentEnc, :PromoIntervalEnc]
lm = lm_for_prediction(all_data, :Promo2SinceYear, features);

In [19]:
apply_lm_to_data(lm, vcat(features, :Promo2SinceYear), :Promo2SinceYearHM);

In [20]:
new_features = [:CompetitionDistanceHM, :CompetitionOpenSinceMonthHM, 
                :CompetitionOpenSinceYearHM, :Promo2SinceWeekHM, :Promo2SinceYearHM];

#### Check Consistence New Features

In [21]:
rmse(yreal, ypred) = sum((yreal - ypred) .^ 2)/length(yreal)
function eval_new_features(data, features)
    results = []
    for feature in features
        data_without_na = data[!isna(data[feature]),:]
        feature_hm = symbol(string(feature, "HM"))
        push!(results, rmse(data_without_na[feature], data_without_na[feature_hm]))
    end
    return DataFrame(feature=features, result=results)
end

eval_new_features (generic function with 1 method)

In [22]:
eval_new_features(all_data, map(f -> symbol(string(f)[1:end-2]), new_features))

,feature,result
1,CompetitionDistance,5.650534221499798e7
2,CompetitionOpenSinceMonth,10.16708048029716
3,CompetitionOpenSinceYear,35.979639297732525
4,Promo2SinceWeek,197.05558247746052
5,Promo2SinceYear,2.731645039233651


### Check Consistence Features Between Train and Test Sets

In [23]:
setdiff(test[:Store], train[:Store])

0-element Array{Int64,1}

### Add New Features

In [35]:
train[:Year] = map(d -> d[1:4], train[:Date])
train[:Month] = map(d -> d[6:7], train[:Date])
train[:Day] = map(d -> d[9:10], train[:Date])

test[:Year] = map(d -> d[1:4], test[:Date])
test[:Month] = map(d -> d[6:7], test[:Date])
test[:Day] = map(d -> d[9:10], test[:Date])

label_year = labelmap(vcat(train[:Year], test[:Year]))
label_month = labelmap(vcat(train[:Month], test[:Month]))
label_day = labelmap(vcat(train[:Day], test[:Day]))

train[:Year] = labelencode(label_year, train[:Year])
train[:Month] = labelencode(label_month, train[:Month])
train[:Day] = labelencode(label_day, train[:Day])

test[:Year] = labelencode(label_year, test[:Year])
test[:Month] = labelencode(label_month, test[:Month])
test[:Day] = labelencode(label_day, test[:Day]);

41088-element Array{Int64,1}:
 15
 16
 17
 18
 19
 20
 21
 22
 23
 24
 25
 26
 27
  ⋮
 20
 21
 22
 23
 24
 25
 26
 27
 28
 29
 30
 31

In [36]:
showcols(train)

1017209x30 DataFrames.DataFrame
| Col # | Name                        | Eltype     | Missing |
|-------|-----------------------------|------------|---------|
| 1     | Store                       | Int64      | 0       |
| 2     | DayOfWeek                   | Int64      | 0       |
| 3     | Date                        | UTF8String | 0       |
| 4     | Sales                       | Int64      | 0       |
| 5     | Customers                   | Int64      | 0       |
| 6     | Open                        | Int64      | 0       |
| 7     | Promo                       | Int64      | 0       |
| 8     | StateHoliday                | UTF8String | 0       |
| 9     | SchoolHoliday               | Int64      | 0       |
| 10    | StoreType                   | UTF8String | 0       |
| 11    | Assortment                  | UTF8String | 0       |
| 12    | CompetitionDistance         | Int64      | 2642    |
| 13    | CompetitionOpenSinceMonth   | Int64      | 323348  |
| 14    | CompetitionOp

In [41]:
final_features = [:Store, :DayOfWeek, :Open, :Promo, :SchoolHoliday, :Promo2,
                  :StateHolidayEnc, :StoreTypeEnc, :AssortmentEnc, :PromoIntervalEnc,
                  :CompetitionDistanceHM, :CompetitionOpenSinceMonthHM, 
                  :CompetitionOpenSinceYearHM, :Promo2SinceWeekHM, :Promo2SinceYearHM, 
                  :Year, :Month, :Day];

## Export Features as Datasets

In [42]:
writetable("data/training_feat_engineering.csv", train[:, vcat(final_features, label)], separator=',')
writetable("data/test_feat_engineering.csv", test[:, vcat(final_features, :Id)], separator=',');;